In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional,Conv1D
from keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import activations
from sklearn.metrics import accuracy_score
url= 'https://drive.google.com/file/d/16-IaHb7TOUaZdFVKlvgiu_1iTyrs7bJ1/view?usp=share_link'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(read_url)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14975 entries, 0 to 14974
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   temperature        14974 non-null  float64
 1   relative humidity  14974 non-null  float64
 2   wind speed         14974 non-null  float64
 3   surface pressure   14974 non-null  float64
 4   rainfall           5442 non-null   float64
dtypes: float64(5)
memory usage: 585.1 KB


,temperature,relative humidity,wind speed,surface pressure,rainfall
0,26.350000,100.00,9.58,101.99,NaN
1,26.440001,100.00,13.72,101.89,NaN
2,27.219999,100.00,9.58,101.67,NaN
3,28.290001,99.50,9.34,100.96,NaN
4,28.059999,100.00,6.50,101.50,NaN
...,...,...,...,...,...
14970,25.550000,95.44,5.11,101.43,NaN
14971,20.240000,87.06,6.42,101.47,NaN
14972,20.770000,93.19,6.66,101.17,NaN
14973,22.070000,93.62,8.09,101.63,NaN


In [2]:
train_split= 0.8
split_idx = int(len(df) * 0.8)
training_set = df.iloc[:,0:3].values
training_set = df[:split_idx].values
test_set = df[split_idx:].values

In [3]:
# 5-day prediction using 30 days data
x_train = []
y_train = []
n_future = 5 #Next 5 days rainfall forecast
n_past = 30 #Past 30 days
for i in range(0, len(training_set) - n_past - n_future + 1):
 x_train.append(training_set[i : i + n_past, 0])
 y_train.append(training_set[i + n_past : i + n_past + n_future, 0])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1))

In [4]:
model = Sequential()
model.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1], 1))))
model.add(Dropout(0.2))
model.add(LSTM(units= 30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units= 30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units= 30))
model.add(Dropout(0.2))
model.add(layers.Dense( units= n_future))
model.add(layers.Activation( activation='relu'))
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])

In [5]:
history = model.fit(x_train,y_train, batch_size = 32,epochs = 500)

Epoch 1/500
374/374 [==============================] - 20s 12ms/step - loss: 631.0668 - accuracy: 0.2578
Epoch 2/500
374/374 [==============================] - 5s 13ms/step - loss: 236.9227 - accuracy: 0.2516
Epoch 3/500
374/374 [==============================] - 5s 14ms/step - loss: 89.2169 - accuracy: 0.2412
Epoch 4/500
374/374 [==============================] - 5s 13ms/step - loss: 43.8656 - accuracy: 0.2241
Epoch 5/500
374/374 [==============================] - 5s 15ms/step - loss: 34.4471 - accuracy: 0.2063
Epoch 6/500
374/374 [==============================] - 5s 13ms/step - loss: 32.6335 - accuracy: 0.2095
Epoch 7/500
374/374 [==============================] - 6s 17ms/step - loss: 32.4841 - accuracy: 0.2081
Epoch 8/500
374/374 [==============================] - 5s 14ms/step - loss: 32.5456 - accuracy: 0.2071
Epoch 9/500
374/374 [==============================] - 5s 14ms/step - loss: 32.3678 - accuracy: 0.2027
Epoch 10/500
374/374 [==============================] - 5s 15ms/step -

In [6]:
x_test = test_set[: n_past, 0]
y_test = test_set[n_past : n_past + n_future, 0]
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (1, x_test.shape[0], 1))
predicted_temperature = model.predict(x_test)
print('Predicted temperature {}'.format(predicted_temperature))
print('Real temperature {}'.format(y_test))

# Print the heatwave array
heatwaves = []
for i in range(len(predicted_temperature[0])):
    if predicted_temperature[0][i] >= y_test[i]:
        heatwaves.append(1)
    else:
        heatwaves.append(0)

print("Heatwave " + str(heatwaves))

total_accuracy = 0
total_weight = sum(range(1, n_future+1))  # weight for each day
for i in range(n_future):
    accuracy = 100 - abs((predicted_temperature[0][i] - y_test[i]) / y_test[i]) * 100
    weight = n_future - i  # weight for this day
    total_accuracy += accuracy * weight
    print("Day {}: {:.2f}%".format(i+1, accuracy))

overall_accuracy = total_accuracy / total_weight
print("Overall Accuracy: {:.2f}%".format(overall_accuracy))

1/1 [==============================] - 2s 2s/step
Predicted temperature [[29.126003 29.046177 28.9765   28.950891 28.958836]]
Real temperature [28.37999916 28.38999939 28.13999939 30.35000038 30.22999954]
Heatwave [1, 1, 1, 0, 0]
Day 1: 97.37%
Day 2: 97.69%
Day 3: 97.03%
Day 4: 95.39%
Day 5: 95.80%
Overall Accuracy: 97.02%
